### Morans I

In [1]:
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
analyze_path = os.path.join(parent_dir, "utils")

os.chdir(analyze_path)

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from utils import get_grid, read_data, read_taiwan_specific
from utils_macro import LocalMoranAnalysis, GetisOrdGiAnalysis

In [7]:
combined_data = read_data()
taiwan, grid_filter = read_taiwan_specific(read_grid=True)
# taiwan.to_file('../ComputedDataV2/Taiwan/taiwan.shp')

# find combined_data which are in taiwan
# combined_data_gdf = gpd.GeoDataFrame(combined_data, 
#                                      geometry=gpd.points_from_xy(combined_data['經度'], 
#                                                                  combined_data['緯度']), crs='EPSG:4326')
# combined_data_gdf.to_crs(epsg=3826, inplace=True)
# combined_data_in_taiwan = gpd.sjoin(combined_data_gdf, taiwan, how='inner', predicate='within')
# combined_data_in_taiwan.to_csv('../ComputedDataV2/Accident/combined_data_in_taiwan.csv', index=False)

In [8]:
used_indices = {idx for sublist in grid_filter['accident_indices'] if isinstance(sublist, list) for idx in sublist}

all_indices = set(combined_data_in_taiwan.index)
missing_indices = all_indices - used_indices
missing_indices_list = sorted(list(missing_indices))

print(f"原始數據總筆數: {len(all_indices)}")
print(f"網格涵蓋總筆數: {len(used_indices)}")
print(f"遺失索引筆數: {len(missing_indices_list)}")

print(f"遺失索引範例: {missing_indices_list[:10]}")

原始數據總筆數: 652328
網格涵蓋總筆數: 652339
遺失索引筆數: 4430
遺失索引範例: [2, 146, 152, 205, 221, 438, 494, 683, 684, 689]


In [ ]:
# hex_grid = get_grid(combined_data, hex_size=0.01, threshold=-1)
# taiwan = taiwan.to_crs(hex_grid.crs)
# hex_grid = hex_grid[hex_grid.intersects(taiwan.unary_union)]
hex_grid = pd.read_csv("../ComputedDataV2/Grid/hex_gridV1.csv")
from shapely import wkt
hex_grid['geometry'] = hex_grid['geometry'].apply(wkt.loads)
hex_grid = gpd.GeoDataFrame(hex_grid, geometry='geometry', crs='EPSG:3826') 

### Infrastructure Analyze

In [ ]:
combined_data['full_infrastructure'] = combined_data['mrt_100m_count'] + combined_data['youbike_100m_count'] + combined_data['parkinglot_100m_count']

In [ ]:
from utils_macro import plot_facility_vs_human_vehicle_subplot

plot_facility_vs_human_vehicle_subplot(
    combined_data,
    facilities=['youbike_100m_count', 'mrt_100m_count', 'parkinglot_100m_count', 'full_infrastructure'],
    accident_col='事故類型及型態大類別名稱',
    accident_type='人與車'
)

### Lisa plot

In [ ]:
lma = LocalMoranAnalysis(hex_grid, taiwan)
lma.calculate_local_moran(best_distance=6, adjacency='queen')
lma.plot_lisa()
lma.lisa_scatter_plot()
# lma.grid.to_csv('../ComputedDataV2/Grid/local_moran_results.csv', index=False)

### Calculate and Plot GI
get_grid因為已經去掉無事故，所以這不會找到冷點

In [ ]:
go = GetisOrdGiAnalysis(hex_grid, taiwan)
go.calculate_gi(best_distance=6, adjacency='queen')
go.plot_gi_map() # grid_giV3 最新的版本，是正確的

### Find the nearest county from grid

In [ ]:
counties = taiwan[['COUNTYNAME', 'geometry']].copy()
hot_hex = hex_grid[hex_grid['hotspot'] != 'Not Significant'].copy()

def find_nearest_county(hexagon, counties_gdf):
    # 每個 hexagon 到所有 county 的距離
    distances = counties_gdf.distance(hexagon)
    nearest_idx = distances.idxmin()
    return counties_gdf.loc[nearest_idx, 'COUNTYNAME']

hot_hex['nearest_county'] = hot_hex['geometry'].apply(lambda x: find_nearest_county(x, counties))
hot_hex['nearest_county'].unique()

### Find the nearest county for each hexagon

In [ ]:
hex_with_county = gpd.sjoin_nearest(hex_grid, counties[['COUNTYNAME', 'geometry']], how='left', distance_col='dist_to_county')
# hex_with_county.to_csv('../ComputedDataV2/Grid/hex_grid_with_county.csv', index=False)

In [ ]:
from utils_macro import hotspot_in_county
from config import category_value_map, feature_name_map, countycity_dct

hotspot_in_county(hot_hex, hex_with_county, countycity_dct, normalize=False, en=False)

主要以道路設計進行，因為天氣、車輛無法討論空間同質

In [ ]:
from utils_macro import attribute_in_city

col = '事故類型及型態子類別名稱'
pivot_sorted = attribute_in_city(combined_data, hot_hex, col, countycity_dct, feature_name_map, category_value_map, en=True)

[<br/>
    '臺北市', '新北市',  # 最北 <br/>
    '桃園市', '新竹市', '新竹縣', '宜蘭縣',  # 北部<br/>
    '苗栗縣', '臺中市', '彰化縣',  # 中部<br/>
    '嘉義市', '嘉義縣', '臺南市', '高雄市', '屏東縣',  # 南部<br/>
    '花蓮縣', '臺東縣'  # 東部<br/>
]<br/>

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns

similarity_matrix = pd.DataFrame(
    cosine_similarity(pivot_sorted.values),
    index=pivot_sorted.index,
    columns=pivot_sorted.index
)

col_en = feature_name_map.get(col, col)
plt.figure(figsize=(10, 8))
sns.heatmap(similarity_matrix, annot=True, cmap='YlGnBu', cbar=False)
plt.title(f'Similarity of {col_en} proportions across cities')
plt.xticks(rotation=30)
plt.show()